In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 16:00:47 WARN Utils: Your hostname, codespaces-e41708 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/04 16:00:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 16:00:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/04 16:00:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.version

'3.5.1'

24/03/04 16:01:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [17]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv/fhv_tripdata_2019-10.csv.gz

--2024-03-04 15:55:03--  https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv/fhv_tripdata_2019-10.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘fhv_tripdata_2019-10.csv.gz’

fhv_tripdata_2019-1     [ <=>                ] 152.56K  --.-KB/s    in 0.05s   

2024-03-04 15:55:04 (3.28 MB/s) - ‘fhv_tripdata_2019-10.csv.gz’ saved [156217]



In [18]:
!gzip -dc fhv_tripdata_2019-10.csv.gz


gzip: fhv_tripdata_2019-10.csv.gz: not in gzip format


In [10]:
!ls -lh data/pq/fhv/2019/10

total 38M
-rw-r--r-- 1 codespace codespace    0 Mar  4 16:02 _SUCCESS
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00000-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00001-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00002-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00003-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00004-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet
-rw-r--r-- 1 codespace codespace 6.3M Mar  4 16:02 part-00005-f6f1ba9b-b56d-4599-a1f6-53cac091c355-c000.snappy.parquet


In [6]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [7]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

df = df.repartition(6)

df.write.parquet('data/pq/fhv/2019/10/')

In [8]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

In [20]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   NULL|                  NULL|
|              B01315|2019-10-05 15:13:04|2019-10-05 15:19:48|         264|          74|   NULL|                B01315|
|              B01984|2019-10-12 17:13:00|2019-10-12 17:40:00|         264|          75|   NULL|                B01984|
|              B00310|2019-10-15 10:55:04|2019-10-15 11:00:45|         264|         247|   NULL|                B03047|
|              B00932|2019-10-08 06:58:42|2019-10-08 07:11:11|         264|          37|   NULL|                B00932|
|              B01029|2019-10-10 14:45:0

In [11]:
from pyspark.sql import functions as F

In [12]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [14]:
df.registerTempTable('fhv_2019_10')

/workspaces/DEZoomcamp2024/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [15]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [16]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [23]:
df \
    .withColumn('duration', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-----------------+
|pickup_date|    max(duration)|
+-----------+-----------------+
| 2019-10-28|         631152.5|
| 2019-10-11|         631152.5|
| 2019-10-31|87672.44083333333|
| 2019-10-01|70128.02805555555|
| 2019-10-17|           8794.0|
+-----------+-----------------+



In [22]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
+-----------+------------------+



In [24]:
spark.sql("""
SELECT
    PULocationID,
    COUNT(1)
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|           2|       1|
|         105|       2|
|         111|       5|
|          30|       8|
|         120|      14|
+------------+--------+



In [25]:
df \
    .groupBy('PULocationID') \
        .count() \
    .orderBy('count') \
    .limit(5) \
    .show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|           2|    1|
|         105|    2|
|         111|    5|
|          30|    8|
|         120|   14|
+------------+-----+



In [26]:
df_zones = spark.read.parquet('zones')

In [27]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [28]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [29]:
df_zones.registerTempTable('zones')

In [35]:
spark.sql("""
SELECT
    pul.Zone,
    PULocationID,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY
    1, 2
ORDER BY
    3 ASC
LIMIT 5;
""").show()

+--------------------+------------+--------+
|                Zone|PULocationID|count(1)|
+--------------------+------------+--------+
|         Jamaica Bay|           2|       1|
|Governor's Island...|         105|       2|
| Green-Wood Cemetery|         111|       5|
|       Broad Channel|          30|       8|
|     Highbridge Park|         120|      14|
+--------------------+------------+--------+

